In [ ]:
import os
import pandas as pd

#System configuration
BATTERY_CAPACITY = 13  #Capacity of the battery in kWh
MAX_CONTINUOUS_POWER = 4.6 #Max charge and discharge in kWh
EFFICIENCY = 0.93  #Charge and discharge efficiency (93 %)
START_BATTERY_LEVEL = 0  #Start value of the battery in kWh
PV_DATA_FOLDER = "pv_data/test"  #Folder of the CSV files
PV_DATA = "pv_data"
EXPORT_FILE = "simulation_report.txt"
TIME_STEP_HOURS = 1  # Time step (1 hour)
ELECTRICITY_COSTS = 0.34 #Costs €/kWh
GRID_FEED_IN_EARNINGS = 0.0958 #Price €/kWh


#New column headers
COLUMN_MAPPING = {
    "PV-Erzeugung / Mittelwerte [W]": "pv_generation",
    "Eigenverbrauch / Mittelwerte [W]": "consumption",
    "Netzbezug / Mittelwerte [W]": "grid_demand"
}

#Functions
def simulate_battery(net_power, battery_level, max_capacity, max_power, efficiency, time_step):
    """
    Simulates the charging and discharging of the battery for a time step.
    :param net_power: Net power (generation - consumption) in the current time step (kWh)
    :param battery_level: Current state of charge of the battery (kWh)
    :param max_capacity: Maximum capacity of the battery (kWh)
    :param max_power: Maximum charging/discharging power (kWh per time step)
    :param efficiency: Charging/discharging efficiency (e.g. 0.93 for 93 %)
    :param time_step: Time step duration in hours (e.g. 1 for 1 hour)
    :return: Tuple (new battery level, stored energy, consumed energy, grid feed-in, grid consumption)
    """
    max_power_per_step = max_power * time_step  #Limit charging/discharging power per time step
    grid_feed_in = 0  #Energy that is fed into the grid
    grid_draw = 0  #Energy drawn from the grid

    if net_power > 0:  #Surplus energy from the PV system
        charge_power = min(net_power, max_power_per_step)  #Charging power limit
        charge_power = min(charge_power, max_capacity - battery_level)  #Limit to available capacity
        actual_charge = charge_power * efficiency  #Taking charging losses into account
        battery_level += actual_charge
        grid_feed_in = max(0, net_power - charge_power)  #Surplus goes to the grid
        return battery_level, actual_charge, 0, grid_feed_in, 0
    else:  # Power requirement, battery supplies energy
        discharge_power = min(-net_power, max_power_per_step)  #Discharge power limit
        discharge_power = min(discharge_power, battery_level)  #Limit to current battery level
        actual_discharge = discharge_power / efficiency  #Take discharge losses into account
        battery_level -= actual_discharge
        unmet_demand = -net_power - discharge_power  #Unmet demand
        grid_draw = max(0, unmet_demand)  #Draw residual energy from the grid
        return battery_level, 0, actual_discharge, 0, grid_draw

def process_daily_data(file_path, battery_level):
    """
    Processes a day's worth of data and simulates battery interaction.
    Aggregates the data to hourly averages.
    :param file_path: Path to the CSV file
    :param battery_level: Current state of charge of the battery (kWh)
    :return: Tuple (new battery level, stored energy, discharged energy, grid feed-in, grid consumption)
    """
    daily_data = pd.read_csv(file_path, delimiter=',')
    
    #Rename columns
    daily_data = daily_data.rename(columns=COLUMN_MAPPING)
    
    #Replacement of NaN values
    daily_data = daily_data.fillna(0)
    
    #Adjust timestamp (starts with 00:15 and has 96 values for 15-minute increments)
    daily_data['Time'] = pd.date_range(start="00:15", periods=96, freq="15T")

    #Hourly aggregation of the mean values (e.g. 00:15-01:00 → 00:15 is grouped to 01:00)
    daily_data = daily_data.resample('H', on='Time').mean()

    stored_energy = 0
    discharged_energy = 0
    total_grid_feed_in = 0
    total_grid_draw = 0

    #Iterate through the hourly aggregated data
    for _, row in daily_data.iterrows():
        pv_power = row['pv_generation']  #Average PV generation (W)
        consumption = row['consumption'] + row['grid_demand']  #Consumption including mains supply (W)
        net_power = (pv_power - consumption) / 1000 * TIME_STEP_HOURS  #Net power in kWh
        battery_level, stored, discharged, grid_feed_in, grid_draw = simulate_battery(
            net_power, battery_level, BATTERY_CAPACITY, MAX_CONTINUOUS_POWER, EFFICIENCY, TIME_STEP_HOURS
        )
        stored_energy += stored
        discharged_energy += discharged
        total_grid_feed_in += grid_feed_in
        total_grid_draw += grid_draw

    return battery_level, stored_energy, discharged_energy, total_grid_feed_in, total_grid_draw


def convert_kw_to_w(input_directory, output_directory):
    """
    This function is used to clean the column headers and convert the values to the required data type.
    """
    #Secures that the output directory does exist
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    #Iterate through all files (only '.csv' files) in the folder
    for filename in os.listdir(input_directory):
        if filename.endswith('.csv'): 
            input_filepath = os.path.join(input_directory, filename)
            output_filepath = os.path.join(output_directory, filename)

            #Loading CSV files in a DataFrame
            df = pd.read_csv(input_filepath, decimal=',', delimiter=';', thousands='.')

            #Checking the column's header
            for column in df.columns:
                if '[kW]' in column:
                    #Convert the vales from kW to W and overwrites the columnes ensuring the format is no decimal value
                    df[column] = pd.to_numeric(df[column], errors='coerce').fillna(0).astype(float) * 1000
                    #Corrects the column header
                    df.rename(columns={column: column.replace('[kW]', '[W]')}, inplace=True)

            #Writes the changed file in an output folder without decimal values
            df.to_csv(output_filepath, index=False)  

            
#Main program
def main():
    battery_level = START_BATTERY_LEVEL
    total_stored_energy = 0
    total_discharged_energy = 0
    total_grid_feed_in = 0
    total_grid_draw = 0
    
    convert_kw_to_w(PV_DATA, PV_DATA_FOLDER)

    #Iterating through all files in the folder
    for file_name in sorted(os.listdir(PV_DATA_FOLDER)):
        file_path = os.path.join(PV_DATA_FOLDER, file_name)
        if file_name.endswith(".csv"):
            battery_level, stored_energy, discharged_energy, grid_feed_in, grid_draw = process_daily_data(file_path, battery_level)
            total_stored_energy += stored_energy
            total_discharged_energy += discharged_energy
            total_grid_feed_in += grid_feed_in
            total_grid_draw += grid_draw
            

    #Return the results
    with open(EXPORT_FILE, "w") as report_file:
        report_file.write("Simulation results for the year: 2024\n")
        report_file.write("\n")
        report_file.write("Technical data:\n")
        report_file.write("Tesla Powerwall 3\n")
        report_file.write(f"Battery capacity: {BATTERY_CAPACITY:.2f} kWh\n")
        report_file.write(f"Charging/discharging capacity: {MAX_CONTINUOUS_POWER:.2f} kWh\n")
        report_file.write(f"Efficiency: {EFFICIENCY:.2f} \n")
        report_file.write("\n")
        report_file.write("Results:\n")
        report_file.write(f"Stored energy: {total_stored_energy:.2f} kWh\n")
        report_file.write(f"Discharged energy: {total_discharged_energy:.2f} kWh\n")
        report_file.write(f"Electricity costs saved: {total_discharged_energy * ELECTRICITY_COSTS:.2f} €\n")
        report_file.write(f"Energy fed into the grid: {total_grid_feed_in:.2f} kWh\n")
        report_file.write(f"Earnings from grid feed-in: {total_grid_feed_in * GRID_FEED_IN_EARNINGS:.2f} €\n")
        report_file.write(f"Energy drawn from the grid: {total_grid_draw:.2f} kWh\n")
        report_file.write(f"Final battery status: {battery_level:.2f} kWh\n")

    print("Simulation completed. Results have been saved in 'simulation_report.txt'.")

#Execute programm
if __name__ == "__main__":
    main()